# Literature Review

A common task while exploring a new topic is to conduct a literature review. In this example we will explore how a multi-agent team can be configured to conduct a _simple_ literature review.

- **Arxiv Search Agent**: Use the Arxiv API to search for papers related to a given topic, write the search results to a file, and generate a set of relevant questions (e.g., number of papers found, authors, publication date, etc.).
- **Report Agent**: Generate a report based on the information collected by the search and stock analysis agents.


First, let us import the necessary modules. 

In [1]:
from autogen_agentchat.agents import CodingAssistantAgent, ToolUseAssistantAgent
from autogen_agentchat.teams.group_chat import RoundRobinGroupChat
from autogen_core.components.models import OpenAIChatCompletionClient
from autogen_core.components.tools import FunctionTool

## Defining Tools 

Next, we will define the tools that the agents will use to perform their tasks. In this case we will define a simple function `search_arxiv` that will use the `arxiv` library to search for papers related to a given topic.  

Finally, we will wrap the functions into a `FunctionTool` class that will allow us to use it as a tool in the agents.

In [2]:

def arxiv_search(query: str, max_results: int = 5) -> list:
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv  

    search = arxiv.Search(
        query=query,
        max_results=max_results,
        sort_by=arxiv.SortCriterion.Relevance
    )
    
    results = []
    for paper in search.results():
        results.append({
            'title': paper.title,
            'authors': [author.name for author in paper.authors],
            'published': paper.published.strftime('%Y-%m-%d'),
            'abstract': paper.summary,
            'pdf_url': paper.pdf_url,
        })
    
    # # Write results to a file
    # with open('arxiv_search_results.json', 'w') as f:
    #     json.dump(results, f, indent=2)
    
    return results

In [3]:
arxiv_search_tool = FunctionTool(arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts")  

## Defining Agents 

Next, we will define the agents that will perform the tasks. 

In [4]:
arxiv_agent = ToolUseAssistantAgent(
    name="Arxiv_Search_Agent",
    tool_schema=[arxiv_search_tool.schema],
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="Search Arxiv for papers related to a given topic, including abstracts", 
)
report_agent = CodingAssistantAgent( 
    name="Report_Agent",
    model_client=OpenAIChatCompletionClient(model="gpt-4o-mini"),
    description="Generate a report based on a given topic", 
)

team = RoundRobinGroupChat( 
    participants=[arxiv_agent,report_agent], 
    tools=[arxiv_search_tool],
)

result = await team.run(task="Write a literature review on AI multiagent systems")
result

--------------------------------------------------------------------------------
user:
Write a literature review on AI multiagent systems
--------------------------------------------------------------------------------
Arxiv_Search_Agent:
[FunctionCall(id='call_VDuOvCNZFJ0h8WkMgaWJjdbj', arguments='{"query":"AI multiagent systems"}', name='arxiv_search')]


/tmp/ipykernel_207909/404270292.py:14: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for paper in search.results():


--------------------------------------------------------------------------------
tool_agent:
[FunctionExecutionResult(content='[{\'title\': \'Human Values in Multiagent Systems\', \'authors\': [\'Nardine Osman\', "Mark d\'Inverno"], \'published\': \'2023-05-04\', \'abstract\': \'One of the major challenges we face with ethical AI today is developing\\ncomputational systems whose reasoning and behaviour are provably aligned with\\nhuman values. Human values, however, are notorious for being ambiguous,\\ncontradictory and ever-changing. In order to bridge this gap, and get us closer\\nto the situation where we can formally reason about implementing values into\\nAI, this paper presents a formal representation of values, grounded in the\\nsocial sciences. We use this formal representation to articulate the key\\nchallenges for achieving value-aligned behaviour in multiagent systems (MAS)\\nand a research roadmap for addressing them.\', \'pdf_url\': \'http://arxiv.org/pdf/2305.02739v1\'}, 

TeamRunResult(messages=[TextMessage(source='user', content='Write a literature review on AI multiagent systems'), StopMessage(source='Arxiv_Search_Agent', content='### Literature Review on AI Multiagent Systems\n\n#### Introduction\nArtificial Intelligence (AI) multiagent systems (MAS) refer to systems composed of multiple interacting intelligent agents. These systems are designed to work collaboratively and make decisions to achieve specific goals. Over the years, the study of MAS has evolved to address complex social problems, improve computational methodologies, and explore human values, contributing significantly to numerous fields such as robotics, social science, and ethics.\n\n#### Recent Advances in AI Multiagent Systems\n\n1. **Human Values in Multiagent Systems**\n   - **Authors:** Nardine Osman, Mark d\'Inverno\n   - **Published:** May 4, 2023\n   - **Abstract:** This paper addresses a critical challenge in ethical AI: aligning computational systems’ reasoning and behavior w